In [1]:
import tensorflow as tf

In [ ]:
def single_step(features, model):
    with tf.GradientTape() as tape:
        projection_head_outputs = model(features, training = True)
        con_loss, logits_con, labels_con = contrastive_loss(
            projection_head_outputs, hidden_norm=FLAGS.hidden_norm, temperature=FLAGS.temperature)
        metrics.update_pretrain_metrics_train(
                contrast_loss_metric, contrast_acc_metric, contrast_entropy_metric,con_loss,logits_con,labels_con)
        weight_decay = model_lib.add_weight_decay(model)
        weight_decay_metric.update_state(weight_decay)
        con_loss += weight_decay
        total_loss_metric.update_state(con_loss)
        grads = tape.gradient(con_loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        

In [4]:
def contrastive_loss(hidden, hidden_norm=True, temperature=1.0):
    if hidden_norm:
        hidden = tf.math.l2_normalize(hidden, -1)
    hidden1, hidden2 = tf.split(hidden, 2, 0)
    batch_size = tf.shape(hidden1)[0]
    labels = tf.one_hot(tf.range(batch_size), batch_size * 2)
    masks = tf.one_hot(tf.range(batch_size), batch_size)
    
    similarity1 = tf.matmul(hidden1, hidden1, transpose_b=True) / temperature
    similarity1 = similarity1 - masks*1e9
    similarity2 = tf.matmul(hidden2, hidden2, transpose_b=True) / temperature
    similarity2 = similarity2 - masks*1e9
    similarity12 = tf.matmul(hidden1, hidden2, transpose_b=True) / temperature
    similarity21 = tf.matmul(hidden2, hidden1, transpose_b=True) / temperature
    
    loss_a = tf.nn.softmax_cross_entropy_with_logits(labels, tf.concat([similarity12, similarity1], 1))
    loss_b = tf.nn.softmax_cross_entropy_with_logits(labels, tf.concat([similarity21, similarity2], 1))
    loss = tf.reduce_mean(loss_a + loss_b)
    
    return loss, similarity12, labels

10

In [15]:
tf.range(16)

<tf.Tensor: shape=(16,), dtype=int32, numpy=
array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15],
      dtype=int32)>

In [17]:
tf.one_hot(tf.range(16), 16)

<tf.Tensor: shape=(16, 16), dtype=float32, numpy=
array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
 

In [13]:
s

<tf.Tensor: shape=(8, 2048), dtype=float32, numpy=
array([[ 0.4967835 ,  0.53118134, -0.74064374, ..., -0.88074803,
        -0.2715497 ,  0.05539417],
       [ 0.9060056 , -0.0352242 ,  0.5805054 , ..., -0.519958  ,
        -0.79685926,  0.26861048],
       [ 0.91540766, -0.55491924, -0.7081678 , ..., -0.6720772 ,
        -0.7275002 , -0.09729791],
       ...,
       [-0.3710258 , -0.27557802, -0.04614735, ...,  0.07753706,
         0.233598  , -0.63305736],
       [-0.77266335,  0.32226324, -0.6943145 , ...,  0.11404729,
         0.07968116,  0.18304658],
       [ 0.48513317,  0.6111293 , -0.5276947 , ...,  0.9431939 ,
        -0.02665091, -0.83411145]], dtype=float32)>

In [14]:
s1

<tf.Tensor: shape=(8, 2048), dtype=float32, numpy=
array([[ 0.6171417 , -0.4228592 ,  0.91086936, ..., -0.74918985,
         0.62310505, -0.8379803 ],
       [-0.7906213 ,  0.6534364 ,  0.85992503, ...,  0.6745951 ,
        -0.9756386 ,  0.8839643 ],
       [-0.7955861 , -0.53152776,  0.26060033, ..., -0.7868061 ,
         0.33414507, -0.23360443],
       ...,
       [-0.15056896, -0.18455172,  0.26310015, ..., -0.9537008 ,
         0.5488591 , -0.34809422],
       [-0.8403487 ,  0.386868  , -0.55087876, ..., -0.28239393,
        -0.04989457,  0.12627053],
       [-0.19365144,  0.23050189, -0.4366145 , ..., -0.38509417,
        -0.80985355, -0.75337005]], dtype=float32)>

In [19]:
train_mode = 'finetune'
fine_tune_after_block = 0

In [20]:
trainable = (train_mode != 'finetune' or fine_tune_after_block == -1)
trainable

False

In [1]:
l1, l2, l3, l4, l5 = ([] for i in range(5))

In [11]:
l5.append(3)
l5[4]

3